In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import requests
import pandas as pd
from io import BytesIO
from datetime import date, timedelta
from sklearn.preprocessing import MinMaxScaler


In [0]:
# Construct the URL for today's forecast file
today = date.today() #+ timedelta(days=1)
url = "https://www.admie.gr/sites/default/files/attached-files/type-file/2023/05/20230508_ISP1Requirements_01.xlsx"

print (url)
# Download the file from the URL
response = requests.get(url)

https://www.admie.gr/sites/default/files/attached-files/type-file/2023/05/20230508_ISP1Requirements_01.xlsx


In [0]:
df = pd.read_excel(BytesIO(response.content), engine='openpyxl', header=None, skiprows=1, index_col=0)

df = df.iloc[[0, 3, 7, 27], :]

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [0]:
df = df.drop(df.columns[[0,49]], axis=1)


In [0]:
df = df.transpose()

In [0]:
new_cols = [col + f"_{i}" for i, col in enumerate(df.columns)]
df.columns = new_cols
df = df.rename(columns={'RES Forecast_0': 'Time', 'Total System_1': 'Total_RES', 'Total System_2': 'Total_ND', 'Total System_3': 'Total_Hydro'})


In [0]:
df['Time'] = pd.to_datetime(df['Time'])
df = df.set_index("Time")

In [0]:
df = df.resample('H').mean()

In [0]:
# add a calculated column 'dams' as the sum of 'A' and 'B'
df = df.assign(ISP=(df['Total_RES'] + df['Total_Hydro'])/df['Total_ND'])

In [0]:
# sort the DataFrame by column 'B'
df = df.sort_values(by='ISP' , ascending = False)

In [0]:
df

,Total_RES,Total_ND,Total_Hydro,ISP
Time,,,,
2023-05-08 12:00:00,3790.0,5430.0,45.0,0.706262
2023-05-08 13:00:00,3650.0,5215.0,15.0,0.702780
2023-05-08 11:00:00,3735.0,5455.0,45.0,0.692942
2023-05-08 14:00:00,3330.0,5010.0,15.0,0.667665
2023-05-08 10:00:00,3450.0,5375.0,47.0,0.650605
2023-05-08 15:00:00,2825.0,4815.0,15.0,0.589823
2023-05-08 09:00:00,2975.0,5295.0,49.0,0.571105
2023-05-08 08:00:00,2305.0,5135.0,150.0,0.478092
2023-05-08 16:00:00,2185.0,4765.0,41.0,0.467156


In [0]:


# Load the forecast and green energy dataframes
forecast_df = pd.read_csv("forecast_bigg.csv")
green_energy_df = df[['ISP']]



In [0]:
#Limit of red tariff zone
threshold_tariff = green_energy_df.iloc[-6,0]

In [0]:
# Convert the Time column to a datetime object
forecast_df["Time"] = pd.to_datetime(forecast_df["Time"])

# Convert the datetime object to the desired format
forecast_df["Time"] = forecast_df["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")


In [0]:
# Reset the index to turn the "Time" index into a regular column
green_energy_df.reset_index(inplace=True)

In [0]:
# Convert the Time column to a datetime object
green_energy_df["Time"] = pd.to_datetime(green_energy_df["Time"])

<command-3216744693488352>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_energy_df["Time"] = pd.to_datetime(green_energy_df["Time"])


In [0]:
# Convert the Time column to a datetime object
forecast_df["Time"] = pd.to_datetime(forecast_df["Time"])

In [0]:
forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774 entries, 0 to 773
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   deviceid     774 non-null    object        
 1   Time         774 non-null    datetime64[ns]
 2   value        774 non-null    float64       
 3   device_type  774 non-null    float64       
 4   home_id      774 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 30.4+ KB


In [0]:
print (forecast_df)
print (green_energy_df)
print (threshold_tariff)

                 deviceid                Time       value  device_type  home_id
0           domxem-C7F92D 2023-05-08 00:00:00   59.632738          1.0     82.0
1           domxem-C7F92D 2023-05-08 01:00:00   49.190871          1.0     82.0
2           domxem-C7F92D 2023-05-08 02:00:00   60.567909          1.0     82.0
3           domxem-C7F92D 2023-05-08 03:00:00   49.720847          1.0     82.0
4           domxem-C7F92D 2023-05-08 04:00:00   54.102657          1.0     82.0
..                    ...                 ...         ...          ...      ...
769  domxem3-FCF5C49641E6 2023-05-08 19:00:00  246.546928          2.0    147.0
770  domxem3-FCF5C49641E6 2023-05-08 20:00:00   40.932821          2.0    147.0
771  domxem3-FCF5C49641E6 2023-05-08 21:00:00   44.957516          2.0    147.0
772  domxem3-FCF5C49641E6 2023-05-08 22:00:00   72.243707          2.0    147.0
773  domxem3-FCF5C49641E6 2023-05-08 23:00:00  100.536754          2.0    147.0

[774 rows x 5 columns]
                

In [0]:
# Merge the two dataframes on time
merged_df = pd.merge(forecast_df, green_energy_df, on="Time")

In [0]:
merged_df

,deviceid,Time,value,device_type,home_id,ISP
0,domxem-C7F92D,2023-05-08 00:00:00,59.632738,1.0,82.0,0.155102
1,domxem-C7FCB5,2023-05-08 00:00:00,144.728927,1.0,81.0,0.155102
2,domxem3-3494546ED134,2023-05-08 00:00:00,110.709881,2.0,199.0,0.155102
3,domxem3-3494546ED524,2023-05-08 00:00:00,13.087863,2.0,275.0,0.155102
4,domxem3-349454717054,2023-05-08 00:00:00,188.367607,2.0,194.0,0.155102
...,...,...,...,...,...,...
769,domxem3-ECFABCC7F0D1,2023-05-08 23:00:00,94.744970,2.0,20.0,0.130435
770,domxem3-ECFABCC7F100,2023-05-08 23:00:00,276.169873,2.0,35.0,0.130435
771,domxem3-ECFABCC7F15D,2023-05-08 23:00:00,704.901137,2.0,37.0,0.130435
772,domxem3-FCF5C49559E1,2023-05-08 23:00:00,53.541722,2.0,129.0,0.130435


In [0]:
merged_df.iloc[:, [2, 5]].values

Out[71]: array([[5.96327376e+01, 1.55102041e-01],
       [1.44728927e+02, 1.55102041e-01],
       [1.10709881e+02, 1.55102041e-01],
       ...,
       [7.04901137e+02, 1.30434783e-01],
       [5.35417219e+01, 1.30434783e-01],
       [1.00536754e+02, 1.30434783e-01]])

In [0]:
# Define the input features and output variable
X = merged_df.iloc[:, [2, 5]].values # Select all columns except deviceid, time, device_type_text and green_energy_value
y = np.where(merged_df["ISP"] <= threshold_tariff, 0, 1) # Set output variable to 0 if green energy value is less than 0.5, otherwise set to 1

In [0]:
X

Out[73]: array([[5.96327376e+01, 1.55102041e-01],
       [1.44728927e+02, 1.55102041e-01],
       [1.10709881e+02, 1.55102041e-01],
       ...,
       [7.04901137e+02, 1.30434783e-01],
       [5.35417219e+01, 1.30434783e-01],
       [1.00536754e+02, 1.30434783e-01]])

In [0]:
y

Out[74]: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [0]:


# Normalize the input features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network architecture
model = Sequential([
    Dense(64, activation="relu", input_dim=X.shape[1]),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Testing accuracy: {accuracy:.2f}")



Epoch 1/10
20/20 [==============================] - 1s 15ms/step - loss: 0.6076 - accuracy: 0.7496 - val_loss: 0.5564 - val_accuracy: 0.7419
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5131 - accuracy: 0.7512 - val_loss: 0.4781 - val_accuracy: 0.7419
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4570 - accuracy: 0.7512 - val_loss: 0.4401 - val_accuracy: 0.7419
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4215 - accuracy: 0.7512 - val_loss: 0.4070 - val_accuracy: 0.7419
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.3892 - accuracy: 0.7512 - val_loss: 0.3754 - val_accuracy: 0.7419
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.3603 - accuracy: 0.7528 - val_loss: 0.3441 - val_accuracy: 0.7419
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.3325 - accuracy: 0.7819 - val_loss: 0.3199 - val_accuracy: 0.8387
Epoch 8/10
20/20 [=

In [0]:
# Use the model to generate the recommendation for each device
merged_df["prediction"] = model.predict(X) # Generate a prediction for each row in the merged dataframe
merged_df["recommendation"] = np.where((merged_df["prediction"] < 0.51) & (merged_df["value"] > 100), "Please avoid using electricity during this period", "") # Generate a recommendation based on the prediction


25/25 [==============================] - 0s 850us/step


In [0]:
model.predict(X)

25/25 [==============================] - 0s 833us/step
Out[77]: array([[0.5031766 ],
       [0.49502897],
       [0.4982861 ],
       [0.50763273],
       [0.4908515 ],
       [0.5049343 ],
       [0.50726587],
       [0.50249135],
       [0.503879  ],
       [0.50539213],
       [0.50452846],
       [0.503497  ],
       [0.50430566],
       [0.49442008],
       [0.5075494 ],
       [0.5303731 ],
       [0.62812686],
       [0.50074816],
       [0.5092928 ],
       [0.501282  ],
       [0.5055588 ],
       [0.5058777 ],
       [0.48775855],
       [0.50332266],
       [0.5042131 ],
       [0.5066359 ],
       [0.5034939 ],
       [0.5015835 ],
       [0.50816876],
       [0.50209635],
       [0.46999365],
       [0.49608386],
       [0.51127774],
       [0.50630987],
       [0.5035736 ],
       [0.51488984],
       [0.5140714 ],
       [0.5117229 ],
       [0.51449347],
       [0.50934964],
       [0.5108687 ],
       [0.51250184],
       [0.511388  ],
       [0.5103522 ],
       [0.51

In [0]:
from sklearn.metrics import confusion_matrix

# Get the predicted binary values
y_pred = model.predict(X)
y_pred = np.where (y_pred<0.51,0,1)

# Create the confusion matrix
cm = confusion_matrix(y, y_pred)

print("Confusion Matrix:")
print(cm)

25/25 [==============================] - 0s 807us/step
Confusion Matrix:
[[187   7]
 [ 16 564]]


In [0]:
model.predict(X)

25/25 [==============================] - 0s 819us/step
Out[98]: array([[0.5031766 ],
       [0.49502897],
       [0.4982861 ],
       [0.50763273],
       [0.4908515 ],
       [0.5049343 ],
       [0.50726587],
       [0.50249135],
       [0.503879  ],
       [0.50539213],
       [0.50452846],
       [0.503497  ],
       [0.50430566],
       [0.49442008],
       [0.5075494 ],
       [0.5303731 ],
       [0.62812686],
       [0.50074816],
       [0.5092928 ],
       [0.501282  ],
       [0.5055588 ],
       [0.5058777 ],
       [0.48775855],
       [0.50332266],
       [0.5042131 ],
       [0.5066359 ],
       [0.5034939 ],
       [0.5015835 ],
       [0.50816876],
       [0.50209635],
       [0.46999365],
       [0.49608386],
       [0.51127774],
       [0.50630987],
       [0.5035736 ],
       [0.51488984],
       [0.5140714 ],
       [0.5117229 ],
       [0.51449347],
       [0.50934964],
       [0.5108687 ],
       [0.51250184],
       [0.511388  ],
       [0.5103522 ],
       [0.51

In [0]:
rec=pd.DataFrame({'Actual': y, 'Pred': y_pred.flatten()})

In [0]:
rec

,Actual,Pred
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
769,0,0
770,0,0
771,0,0
772,0,0


In [0]:
recco = merged_df[merged_df['recommendation']!='']

In [0]:
recco['deviceid'].unique()

Out[109]: array(['domxem-C7FCB5', 'domxem3-3494546ED134', 'domxem3-349454717054',
       'domxem3-84CCA8ACA770', 'domxem3-ECFABCC7EED2',
       'domxem3-FCF5C49559E1', 'domxem3-FCF5C49641E6',
       'domxem3-C82B96116354', 'domxem3-ECFABCC7EE50',
       'domxem3-485519C9AD0E', 'domxem3-ECFABCC7EF31',
       'domxem3-68C63AFADDBF', 'domxem3-68C63AFADE3A',
       'domxem3-68C63AFB1F30', 'domxem3-C8C9A370578A',
       'domxem3-ECFABCC7EE1B', 'domxem3-ECFABCC7F05A',
       'domxem3-ECFABCC7F071', 'domxem3-ECFABCC7F0D1',
       'domxem3-ECFABCC7F100', 'domxem3-ECFABCC7F15D'], dtype=object)

In [0]:
recco[recco['deviceid']=="domxem3-ECFABCC7EED2"]

,deviceid,Time,value,device_type,home_id,ISP,prediction,recommendation
22,domxem3-ECFABCC7EED2,2023-05-08 00:00:00,220.684859,2.0,24.0,0.155102,0.487759,Please avoid using electricity during this period
54,domxem3-ECFABCC7EED2,2023-05-08 01:00:00,362.680163,2.0,24.0,0.156559,0.481272,Please avoid using electricity during this period
86,domxem3-ECFABCC7EED2,2023-05-08 02:00:00,363.376578,2.0,24.0,0.155172,0.474476,Please avoid using electricity during this period
118,domxem3-ECFABCC7EED2,2023-05-08 03:00:00,424.976970,2.0,24.0,0.158046,0.482700,Please avoid using electricity during this period
150,domxem3-ECFABCC7EED2,2023-05-08 04:00:00,216.854357,2.0,24.0,0.150838,0.467374,Please avoid using electricity during this period
182,domxem3-ECFABCC7EED2,2023-05-08 05:00:00,383.671193,2.0,24.0,0.152010,0.457480,Please avoid using electricity during this period
731,domxem3-ECFABCC7EED2,2023-05-08 22:00:00,868.224563,2.0,24.0,0.128682,0.383181,Please avoid using electricity during this period
764,domxem3-ECFABCC7EED2,2023-05-08 23:00:00,797.205671,2.0,24.0,0.130435,0.385353,Please avoid using electricity during this period


In [0]:
# Select only necessary columns
recco_df = pd.DataFrame(recco, columns=['deviceid', 'Time', 'recommendation'])


In [0]:
merged_df[merged_df['deviceid']=="domxem3-ECFABCC7EE50"]

,deviceid,Time,value,device_type,home_id,ISP,prediction,recommendation
20,domxem3-ECFABCC7EE50,2023-05-08 00:00:00,34.752360,2.0,1.0,0.155102,0.505559,
52,domxem3-ECFABCC7EE50,2023-05-08 01:00:00,139.777226,2.0,1.0,0.156559,0.502606,Please avoid using electricity during this period
84,domxem3-ECFABCC7EE50,2023-05-08 02:00:00,88.377343,2.0,1.0,0.155172,0.500768,
116,domxem3-ECFABCC7EE50,2023-05-08 03:00:00,52.454352,2.0,1.0,0.158046,0.518213,
148,domxem3-ECFABCC7EE50,2023-05-08 04:00:00,32.504735,2.0,1.0,0.150838,0.484983,
180,domxem3-ECFABCC7EE50,2023-05-08 05:00:00,38.859892,2.0,1.0,0.152010,0.490089,
212,domxem3-ECFABCC7EE50,2023-05-08 06:00:00,46.171387,2.0,1.0,0.226107,0.802657,
244,domxem3-ECFABCC7EE50,2023-05-08 07:00:00,32.068875,2.0,1.0,0.344191,0.970576,
276,domxem3-ECFABCC7EE50,2023-05-08 08:00:00,54.003610,2.0,1.0,0.478092,0.996194,
308,domxem3-ECFABCC7EE50,2023-05-08 09:00:00,28.025449,2.0,1.0,0.571105,0.999093,


In [0]:
recco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 1 to 773
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   deviceid        57 non-null     object
 1   Time            57 non-null     object
 2   recommendation  57 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [0]:
import json

In [0]:
# Group the filtered dataframe by deviceid
grouped_df = recco_df.groupby("deviceid")

In [0]:
# Create a dictionary with deviceid as key and a list of hours to avoid as value
result = {}
for deviceid, group in grouped_df:
    hours_to_avoid = list(group["Time"].dt.strftime('%Y-%m-%d %H:%M:%S'))
    print(hours_to_avoid)
    result[deviceid] = hours_to_avoid

print (result)
# Convert the dictionary to a JSON string
json_str = json.dumps(result)

print(json_str)




['2023-05-08 00:00:00', '2023-05-08 01:00:00', '2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 00:00:00', '2023-05-08 01:00:00', '2023-05-08 02:00:00', '2023-05-08 04:00:00', '2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 00:00:00']
['2023-05-08 05:00:00']
['2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 22:00:00']
['2023-05-08 00:00:00', '2023-05-08 01:00:00', '2023-05-08 02:00:00', '2023-05-08 04:00:00', '2023-05-08 05:00:00', '2023-05-08 21:00:00', '2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 01:00:00']
['2023-05-08 22:00:00']
['2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 01:00:00']
['2023-05-08 00:00:00', '2023-05-08 01:00:00', '2023-05-08 02:00:00', '2023-05-08 03:00:00', '2023-05-08 04:00:00', '2023-05-08 05:00:00', '2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 21:00:00', '2023-05-08 22:00:00']
['2023-05-08 22:00:00', '2023-05-08 23:00:00']
['2023-05-08 22:00:

In [0]:

# json_str contains the JSON string to be written to file
with open("forecast_recco_bigg.json", "w") as outfile:
    outfile.write(json_str)
